In [2]:
from gradio_client import Client
from pathlib import Path
import json
import pandas as pd

/Users/jimmy/Documents/GitHub/LLM-Leaderboard-Integration/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
path_data = Path("../data")
path_llm = path_data / "llm"
path_lvlm = path_data / "lvlm"

In [4]:
client = Client("https://felixz-open-llm-leaderboard.hf.space/")
json_data = client.predict("","", api_name='/predict')

with open(json_data, 'r') as file:
    file_data = file.read()
    data = json.loads(file_data)
    df = pd.DataFrame(data['data'], columns=data['headers'])
    df.drop(columns=['Model'], inplace=True)
    df.to_json(path_llm / 'HuggingFace-Open-llm-leaderboard-20231116.json', orient='records', indent=4)

Loaded as API: https://felixz-open-llm-leaderboard.hf.space/ ✔


In [84]:
df = pd.read_csv(path_llm / 'a.csv')
df.to_json(path_llm / 'Sherlock.json', orient='records', indent=4)
len(df)

96

In [ ]:
'Text-to-Text Generation', 'Text-to-Image Generation', 'Text-to-Code Generation', 'Text-to-Audio Generation', 'Text-to-Video Generation', 'Image-to-Text Generation', 'Code-to-Text Generation', 'Audio-to-Text Generation', 'Video-to-Text Generation', 'Image-to-Image Generation', 'Code-to-Code Generation', 'Audio-to-Audio Generation', 'Video-to-Video Generation',

'Code Summarization', 'Code Review', 'Identifier Prediction', 'Defect Detection', 'Clone Detection', 'Code Classification', 'Code Reasoning', 'Document Translation', 'Log Parsing',

# 'Image Captioning', 'Sign Language Recognition', 'Emotion Recognition', 'Video Processing', 'Digital Human', 'Multimodality'

In [89]:
labels = [
    "Natural Language Understanding",
    "Natural Language Inference",
    "Question Answering",
    "Summarization",
    "Translation",
    "Sentiment Analysis",
    "Dialogue",
    "Text Classification",
    "Information Retrieval",
    "Knowledge Completion",
    "Relation Extraction",
    "Ethics and Morality",
    "Societal Bias",
    "Toxicity",
    "Fairness",
    "Hallucination",
    "Calibration",
    "Efficiency",
    "Sequence Tagging",
    "Coreference Resolution",
    "Fact Extraction",
    "Multilinguality",
    "Hate Detection",
    "Recommendation",
    "Healthcare",
    "Education",
    "Honestness",
    "Helpfulness",
    "Harmlessness",
    "Robustness",
    "Quality",
    "Alignment",
    "Aesthetics",
    "Originality",
    "Fidelity",
    "Risk",
    "Law",
    "Mathematics",
    "Social Science",
    "Natural Science",
    "Finance",
    "Engineering",
    "Truthfulness",
    "Code Reasoning",
    "Commonsense Reasoning",
    "Knowledge Reasoning",
    "Multi-hop Reasoning",
    "Logical Reasoning",
    "Arithmetic Reasoning",
    "Symbolic Reasoning",
    "Attribute Reasoning",
    "Relation Reasoning",
    "Tool Creation",
    "Tool Manipulation",
    "Robotic Tasks",
    "Code Executor",
    "Calculator",
    "Search Engine",
    "Online Shopping",
    "Personality Testing",
    "Crowd-sourcing Testing",
    "Human-in-the-loop",
]

labels2 = [
    "Visual Perception",
    "Visual Reasoning",
    'Visual Knowledge Acquisition',
    "Visual Commonsense",
    "Object Hallucination",
    "Embodied Intelligence",
]

# "Open Book",
# "Closed Book",
# 'Single Choice',
# 'Multiple Choice',

benchmark_lvlm_labels = {
    'A-OKVQA': [
        "Perception",
        "Existence",
        "Position",
        "Color"
    ],
    'ALFWorld': [
        "Perception",
        "Existence",
        "Position",
        "Color"
    ]
}

benchmark_llm_labels = {
    '2WikiMultihopQA': [
        "Question Answering",
        "Multi-hop Reasoning",
        "Information Retrieval",
        "Knowledge Reasoning",
        "Commonsense Reasoning",
        "Natural Language Inference",
        "Relation Extraction"
    ],
    'A-OKVQA': [
        "Question Answering",
        "Commonsense Reasoning",
        "Knowledge Reasoning",
        "Relation Reasoning"
    ],
    'AFQMC': [
        "Natural Language Understanding",
        "Text Classification",
        "Natural Language Inference",
        "Commonsense Reasoning",
        "Finance"
    ],
    'AGIEval': [
        "Natural Language Understanding",
        "Question Answering",
        "Text Classification",
        "Knowledge Reasoning",
        "Logical Reasoning",
        "Arithmetic Reasoning",
        "Multi-hop Reasoning"
    ],
    'ALFWorld': [
        "Knowledge Reasoning",
        "Relation Reasoning",
        "Tool Manipulation",
        "Robotic Tasks",
        "Commonsense Reasoning"
    ],
    'AlpacaEval': [
        "Natural Language Understanding",
        "Dialogue",
        "Text Classification",
        "Information Retrieval",
        "Knowledge Completion",
        "Helpfulness",
        "Robustness",
        "Quality"
    ],
    'Amazon Review': [
        "Sentiment Analysis",
        "Text Classification",
        "Information Retrieval",
        "Recommendation",
        "Summarization",
        "Helpfulness"
    ],
    'ANGO': [
        
    ]
}

58

In [87]:
from openpyxl import load_workbook, Workbook

# Path to your Excel file and output file
input_file = 'a.xlsx'
output_file = 'b.xlsx'

# Load the workbook and select the active worksheet
wb = load_workbook(input_file)
ws = wb.active

# Dictionary to store unique values and their hyperlinks
unique_values = {}

# Loop through all cells in the worksheet
for row in ws.iter_rows():
    for cell in row:
        # Check if cell has a value
        if cell.value:
            # Store value and hyperlink (if any)
            unique_values[cell.value] = cell.hyperlink.target if cell.hyperlink else None

# Create a new workbook and select the active worksheet
new_wb = Workbook()
new_ws = new_wb.active

# Write unique values and hyperlinks to the new worksheet
for i, (text, hyperlink) in enumerate(unique_values.items(), start=1):
    new_ws.cell(row=i, column=1, value=text)
    if hyperlink:
        new_ws.cell(row=i, column=1).hyperlink = hyperlink

# Save the new workbook
new_wb.save(output_file)

print(f"Unique values with hyperlinks written to {output_file}")


Unique values with hyperlinks written to b.xlsx


In [ ]:
benchmark_links = {
    'ALFWorld': 'https://github.com/alfworld/alfworld',
    'BoolQ': 'https://github.com/google-research-datasets/boolean-questions',
    'Chatbot Arena Conversations': 'https://huggingface.co/datasets/lmsys/chatbot_arena_conversations',
    'CMMLU': 'https://github.com/haonan-li/CMMLU',
    'GAOKAO-Bench-2023': 'https://github.com/OpenLMLab/GAOKAO-Bench-2023',
    'HumanEval': 'https://github.com/openai/human-eval',
    'IMDB': 'https://huggingface.co/datasets/imdb',
    'Mind2Web': 'https://github.com/OSU-NLP-Group/Mind2Web',
    'MMLU': 'https://github.com/hendrycks/test',
    'MT-Bench': 'https://paperswithcode.com/dataset/mt-bench',
    'MultiPL-E': 'https://github.com/nuprl/multipl-e',
    'RAFT': 'https://huggingface.co/datasets/ought/raft',
    'TruthfulQA': 'https://github.com/sylinrl/TruthfulQA',
    'WebShop': 'https://github.com/princeton-nlp/webshop',  
}

benchmark_tags = {
    'ALFWorld': 'Aligning Text and Embodied Environments for Interactive Learning',
    'BoolQ': 'https://github.com/google-research-datasets/boolean-questions',
    'Chatbot Arena Conversations': 'https://huggingface.co/datasets/lmsys/chatbot_arena_conversations',
    'CMMLU': 'https://github.com/haonan-li/CMMLU',
    'GAOKAO-Bench-2023': 'https://github.com/OpenLMLab/GAOKAO-Bench-2023',
    'HumanEval': 'https://github.com/openai/human-eval',
    'IMDB': 'https://huggingface.co/datasets/imdb',
    'Mind2Web': 'https://github.com/OSU-NLP-Group/Mind2Web',
    'MMLU': 'https://github.com/hendrycks/test',
    'MT-Bench': 'https://paperswithcode.com/dataset/mt-bench',
    'MultiPL-E': 'https://github.com/nuprl/multipl-e',
    'RAFT': 'https://huggingface.co/datasets/ought/raft',
    'TruthfulQA': 'https://github.com/sylinrl/TruthfulQA',
    'WebShop': 'https://github.com/princeton-nlp/webshop',  
}

In [46]:
text = '''ALFWorld	WebShop	Mind2Web																																																										
MultiPL-E	HumanEval																																																											
MMLU	MT-Bench	Chatbot Arena Conversations																																																										
BoolQ	MMLU	TruthfulQA	IMDB	RAFT	CMMLU	GAOKAO-Bench-2023	CSL	CHID	CLUEWSC	LLSRC	SLSRC	SLPWC	SLRFC	EPRSTMT	TNEWS	OCNLI	BUSTM																																											
MMLU	gnad10	HellaSwag	ARC	Belebele	XNLI	Amazon Review																																																						
Caltech-UCSD Birds-200-2011	Common Syntactic Processes	Demographic Stereotypes	PaintSkills	I2P	MS-COCO	Magazine Cover Photos	Mental Disorders	P2	Relational Understanding	TIME	Winoground	dailydall.e																																																
APPS	BBQ	BLiMP	BOLD	BoolQ	CNN/DailyMail	CivilComments	Data imputation	Disinformation	Dyck	Entity matching	GSM8K	HellaSwag	HumanEval	ICE	IMDB	AR-LSAT	LegalSupport	MATH	MMLU	MS MARCO	NarrativeQA	NaturalQuestions	OpenbookQA	QuAC	RAFT	RealToxicityPrompts	LIME	WikiFact	XSUM	bAbI	Synthetic efficiency	The Pile	TruthfulQA	TwitterAAE																										
ARC	HellaSwag	MMLU	TruthfulQA	WinoGrande	GSM8K	DROP																																																						
MMLU	BBH	DROP	CRASS	HumanEval	IMPACT	HHH																																																						
MBPP	Spider	NL2Bash																																																										
AR	ER	NER	JS	CR	CFM	SCM	CJP	CTP	LQA	JRG	CU	LC																																																
CIFAR-10	Flickr30k	VOC2012	Omnibenchmark	FSC147	ScienceQA	MMBench	SEED-Bench	MS-COCO																																																				
Chatbot-Arena	HellaSwag	HumanEval	LAMBADA	MMLU	TriviaQA	WinoGrande																																																						
HotpotQA	2WikiMultihopQA	MuSiQue	DuReader	MultiFieldQA	NarrativeQA	Qasper	GovReport	QMSum	Multi-News	VCSUM	TriviaQA	SAMSum	TREC	LSHTC	PassageRetrieval	PassageCount	LCC	RepoBench-P																																										
NarrativeQA	MCScript	CosmosQA	SIQA	DROP	Quoref																																																							
HellaSwag	MMLU	ARC	TruthfulQA	CommonGen																																																								
C-Eval	MMLU	ARC	CHID	WiC	GAOKAO-Bench	CMMLU	AGIEval	AFQMC	WSC	TyDiQA	Flores	BoolQ	CommonSenseQA	C3	RACE	OpenbookQA	CSL	LCSTS	XSum	EPRSTMT	LAMBADA	CMNLI	OCNLI	AX-b	AX-g	RTE	ReCoRD	HellaSwag	PIQA	SIQA	MATH	GSM8K	DROP	HumanEval	MBPP	BBH																								
M3KE	MMLU	GAOKAO-Bench	TGEA	OL-CC	CSNLI	CLUEWSC	ChineseSquad	CHID	WPLC	BiPaR	CommonMT	C3	CMNLI	TOCP	SWSR	CORGI-PM	CDIAL-BIAS	COLD	CBBQ	TUMCC	Cooridinate AI	Corrigible	Myopia Reward	One-box Tendency	Power-seeking	Self-awareness	CAIL判决预测数据集																																	
KQApro	LC-quad2	WQSP	CWQ	GrailQA	GraphQ	QALD-9	MKQA	FewNERD	FewRel	InstructIE	MAVEN	WikiEvents																																																
Flowers102	CIFAR10	ImageNet-1K	Pets37	VizWiz-yesno	VizWiz-singleChoice	TDIUC-Sport	TDIUC-Scene	MEDIC	MSCOCO-MCI	MSCOCO-GOI	MSCOCO-MOS	TDIUC-Color	TDIUC-Utility	TDIUC-Position	TDIUC-Detection	TDIUC-Counting	RefCOCO	MSCOCO-OC	VQA v2	GQA	Whoops	OK-VQA	ScienceQA	VizWiz	ViQuAE	K-ViQuAE	A-OKVQA	A-OKVQRA	A-OKVQAR	ImageNetVC	CLEVR	VSR	MP3D	VQA-MT	VisDial	MSCOCO-ITM	MSCOCO-ITS	WikiHow	Winoground	SNLI-VE	MOCHEG	Grounded IC15	IC15	Grounded COCO-Text	COCO-Text	Grounded TextOCR	TextOCR	CUTE80	IIIT5K	WordArt	FUNSD	POIE	SROIE	TextVQA	DocVQA	OCR-VQA	MSCOCO	TextCaps	NoCaps	Flickr30K
BoolQ	CommitmentBank	COPA	MultiRC	ReCoRD	RTE	WiC	WSC	Broadcoverage Diagnostics	Winogender																																																			
MMLU	TriviaQA	NaturalQuestions	GSM8K	HumanEval	AGIEval	BoolQ	HellaSwag	OpenbookQA	QuAC	WinoGrande'''

from collections import defaultdict
df = defaultdict(int)
for items in text.split('\n'):
    for item in items.split():
        df[item] += 1

sorted(df.items(), key=lambda x: x[1], reverse=True)

[('MMLU', 11),
 ('HellaSwag', 7),
 ('HumanEval', 6),
 ('BoolQ', 5),
 ('TruthfulQA', 4),
 ('ARC', 4),
 ('GSM8K', 4),
 ('DROP', 4),
 ('CHID', 3),
 ('NarrativeQA', 3),
 ('OpenbookQA', 3),
 ('WinoGrande', 3),
 ('TriviaQA', 3),
 ('Grounded', 3),
 ('IMDB', 2),
 ('RAFT', 2),
 ('CMMLU', 2),
 ('CSL', 2),
 ('CLUEWSC', 2),
 ('EPRSTMT', 2),
 ('OCNLI', 2),
 ('MS-COCO', 2),
 ('Winoground', 2),
 ('MATH', 2),
 ('NaturalQuestions', 2),
 ('QuAC', 2),
 ('BBH', 2),
 ('MBPP', 2),
 ('ScienceQA', 2),
 ('LAMBADA', 2),
 ('SIQA', 2),
 ('WiC', 2),
 ('GAOKAO-Bench', 2),
 ('AGIEval', 2),
 ('WSC', 2),
 ('C3', 2),
 ('CMNLI', 2),
 ('RTE', 2),
 ('ReCoRD', 2),
 ('IC15', 2),
 ('COCO-Text', 2),
 ('TextOCR', 2),
 ('ALFWorld', 1),
 ('WebShop', 1),
 ('Mind2Web', 1),
 ('MultiPL-E', 1),
 ('MT-Bench', 1),
 ('Chatbot', 1),
 ('Arena', 1),
 ('Conversations', 1),
 ('GAOKAO-Bench-2023', 1),
 ('LLSRC', 1),
 ('SLSRC', 1),
 ('SLPWC', 1),
 ('SLRFC', 1),
 ('TNEWS', 1),
 ('BUSTM', 1),
 ('gnad10', 1),
 ('Belebele', 1),
 ('XNLI', 1),
 ('Am

In [78]:
metrics = '''10-bin expected calibration error
1-bin expected calibration error
Max prob
10-bin expected calibration error
1-bin expected calibration error (after Platt scaling)
10-bin Expected Calibration Error (after Platt scaling)
Platt Scaling Coefficient
Platt Scaling Intercept
Selective coverage-accuracy area
Accuracy at 10% coverage
Stereotypical associations (race, profession)
Stereotypical associations (gender, profession)
Demographic representation (race)
Demographic representation (gender)
Toxic fraction
Denoised inference runtime (s)
Estimated training emissions (kg CO2)
Estimated training energy cost (MWh)
Observed inference runtime (s)
Idealized inference runtime (s)
Denoised inference runtime (s)
# trials
# prompt tokens
# output tokens
# eval
# train
truncated
SummaC
QAFactEval
Coverage
Density
Compression
BERTScore (F1)
HumanEval-faithfulness
HumanEval-relevance
HumanEval-coherence
Avg. # tests passed
Strict correctness
BBQ (ambiguous)
BBQ (unambiguous)
Longest common prefix length
Edit distance (Levenshtein)
Edit similarity (Levenshtein)
Self-BLEU
Entropy (Monte Carlo)
Macro-F1
Micro-F1
Chinese iBLEU
Chinese Top-1 Accuracy
Chinese ROUGE-2 score
Chinese BLEU-1 score
CLEVA Math Exact Match
Chinese BLEU-1 score
Chinese BLEU-1 score'''

# s = set()
df = pd.DataFrame()
for scenario in metrics.split('\n'):
    # scenario_ = scenario.split('(')[0].strip()
    s = {
        'Name': scenario
    }
    df = pd.concat([df, pd.DataFrame([s])], ignore_index=True)
    # s.add(scenario)
# len(df)
df.to_csv('metrics.csv', index=False)

In [ ]:
# scenarios = '''BoolQ
# NarrativeQA
# NaturalQuestions (closed-book)
# NaturalQuestions (open-book)
# QuAC (Question Answering in Context)
# HellaSwag
# OpenbookQA
# TruthfulQA
# MMLU (Massive Multitask Language Understanding)
# MS MARCO (regular track)
# MS MARCO (TREC track)
# CNN/DailyMail
# XSUM
# IMDB
# CivilComments
# RAFT (Real-world Annotated Few-Shot)
# ICE (International Corpus of English)
# The Pile
# TwitterAAE
# BLiMP (The Benchmark of Linguistic Minimal Pairs for English)
# NaturalQuestions (closed-book)
# HellaSwag
# OpenbookQA
# TruthfulQA
# MMLU (Massive Multitask Language Understanding)
# WikiFact
# bAbI
# Dyck
# Synthetic reasoning (abstract symbols)
# Synthetic reasoning (natural language)
# GSM8K (Grade school math word problems)
# MATH
# MATH (chain-of-thoughts)
# APPS (Code)
# HumanEval (Code)
# LegalSupport
# LSAT
# Data imputation
# Entity matching
# Copyright (text)
# Copyright (code)
# Disinformation (reiteration)
# Disinformation (wedging)
# BBQ (Bias Benchmark for Question Answering)
# BOLD (Bias in Open-Ended Language Generation Dataset)
# RealToxicityPrompts
# Synthetic efficiency
# MMLU (Massive Multitask Language Understanding)
# IMDB
# RAFT (Real-world Annotated Few-Shot)
# CivilComments
# NaturalQuestions (open-book)
# CNN/DailyMail
# IMDB
# CivilComments
# HellaSwag
# OpenbookQA
# TruthfulQA
# MMLU (Massive Multitask Language Understanding)
# BLiMP (The Benchmark of Linguistic Minimal Pairs for English)
# LegalSupport
# LSAT
# BBQ (Bias Benchmark for Question Answering)
# NaturalQuestions (open-book)
# CNN/DailyMail
# IMDB
# CivilComments
# BoolQ
# IMDB'''
# s = set()
# for scenario in scenarios.split('\n'):
#     scenario_ = scenario.split('(')[0].strip()
#     s.add(scenario_)
# # len(s)
# s

In [ ]:
# scenarios = '''MS-COCO (base)
# Caltech-UCSD Birds-200-2011
# DrawBench (image quality categories)
# DrawBench (reasoning categories)
# DrawBench (knowledge categories)
# PartiPrompts (image quality categories)
# PartiPrompts (reasoning categories)
# PartiPrompts (knowledge categories)
# MS-COCO (base)
# DrawBench (image quality categories)
# PartiPrompts (image quality categories)
# MS-COCO (base)
# MS-COCO (Art styles)
# dailydall.e
# Logos
# Landing Page
# Magazine Cover Photos
# dailydall.e
# Logos
# Landing Page
# Magazine Cover Photos
# Common Syntactic Processes
# DrawBench (reasoning categories)
# PartiPrompts (reasoning categories)
# Relational Understanding
# Detection (PaintSkills)
# Winoground
# TIME
# DrawBench (knowledge categories)
# PartiPrompts (knowledge categories)
# Demographic Stereotypes
# Mental Disorders
# Inappropriate Image Prompts (I2P)
# MS-COCO (fairness - AAVE dialect)
# MS-COCO (fairness - gender)
# MS-COCO (robustness)
# MS-COCO (Chinese)
# MS-COCO (Hindi)
# MS-COCO (Spanish)
# MS-COCO Fidelity
# MS-COCO Efficiency
# MS-COCO (Art styles)'''
# s = set()
# for scenario in scenarios.split('\n'):
#     scenario = scenario.split('(')[0].strip()
#     s.add(scenario)
# s

In [7]:
import glob

sum = 0

for file in glob.glob(str(path_llm / "*.json")):
    if 'imagenet' not in file.lower():
        continue
    df = pd.read_json(file)
    sum += len(df)
sum

1223

In [97]:
df = pd.read_json('/Users/jimmy/Downloads/LMExamQA.json')
df['domain'].nunique()

1000

In [63]:
json_files = path_llm.glob('*.json')

# Function to read a JSON file into a DataFrame
def read_json_file(file_path):
    return pd.read_json(file_path)

# Function to find a column that contains 'model' in its name
def find_model_column(df):
    for col in df.columns:
        if 'model' in col.lower():
            return col
    return None

dataframes = []
# Read each file and store in a list of DataFrames
for file in json_files:
    if 'HELM' in file.name:
        continue
    dataframes.append(read_json_file(file))

# Extract and count models in each DataFrame
model_counts = []
for i, df in enumerate(dataframes):
    model_col = find_model_column(df)
    if (model_col is not None) and (type(df[model_col][0]) == str):
        count = df[model_col].fillna('').apply(lambda x: x.split('\n')[0]).value_counts().reset_index()
        count.columns = ['model', 'count']
        model_counts.append(count)

combined_df = pd.concat(model_counts, ignore_index=True)
combined_df.groupby('model').sum().sort_values('count', ascending=False).head(10)


,count
model,
ChatGLM2-6B,41
GPT-4,34
bloom-7b1,33
llama-7B,31
Baichuan2-13B-Chat,31
ChatGPT,30
Qwen-7B-Chat,27
ImageLLM,27
Qwen-14B-Chat,25
